In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introducción

### Contexto
Es importante para las compañias el poder detectar transacciones bancarias fraudulentas para que así sus clientes no tengan cargos por productos que no han comprado.

### El set de datos
El conjunto de datos contiene transacciones hechas por tarjetas de crédito en septiembre de 2013 por tarjetahabientes de Europa.

### Un problema común
En total, el set de datos contiene 284,807 transacciones, pero de estas, solo 492 son fraudulentas, es decir, el set está increiblemente desbalanceado. Por lo que al utilizar modelos de aprendizaje máquina habituales, corremos el riesgo de que este no logre aprender y generalizar las características fundamentales de cada clase para diferenciar entre transacciones reales y fraudulentas.

### Acerca de las columnas
Las columnas contienen solamente datos numéricos, obtenidos a traves de una transformacion PCA (Principal Component Analysis ó Análisis de Componentes Principales). Desafortunadamente, debido a que las características originales revelan información privada de los clientes, no podemos conocerlas con exactitud.
Las caracteristicas V1, V2, ... V28 son los componentes principales obtenidos con PCA. Las columnas 'Tiempo' y 'Monto' son las únicas que no han sido transformadas. 'Tiempo' contiene los segundos tomados entre cada transacción y la primera transacción del set de datos. 'Monto' es el monto de la transacción. La característica 'Clase' toma el valor 1 en caso de fraude y 0 en otro caso.

### Qué esperar en esta libreta
- Métodos para balancear los datos.
- Comparación entre distintos métodos tradicionales de aprendizaje máquina.
- Una aproximación al problema utilizando autoencoders.

### Créditos
#### Dataset
https://www.kaggle.com/mlg-ulb/creditcardfraud
#### Ideas
Algunas ideas fueron tomadas de los siguientes enlaces
- [Este video](https://www.youtube.com/watch?v=NCgjcHLFNDg)
- [Esta libreta](https://www.kaggle.com/robinteuwens/anomaly-detection-with-auto-encoders)

Libreta realizada originalmente en [kaggle](kaggle.com) por Guillermo Velazquez

# Importando las librerías necesarias

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# random
import random

In [3]:
# cargar el set de datos
data_path = '../input/creditcardfraud/creditcard.csv' # La ruta al set de datos en el la libreta de kaggle
df = pd.read_csv(data_path)

# Semillas aleatorias para garantizar la reproducibilidad de los datos
SEMILLA_ALEATORIA = 11  # Puede ser cualquier número
MUESTRA_ENTRENAMIENTO = 200000 # El número de muestras que tomaremos para entrenar

np.random.seed(SEMILLA_ALEATORIA)
random.seed(SEMILLA_ALEATORIA)
tf.random.set_seed(SEMILLA_ALEATORIA)

## Renombrando las columnas
Los nombres de las columnas están en inglés e inician con mayusculas. Asi que vamos a convertirlas para evitar errores de mano y para que todo esté en español.

In [4]:
df.head()

In [5]:
df.columns = map(str.lower, df.columns)
df.rename(columns={'time':'tiempo', 'amount':'monto', 'class':'fraude'}, inplace=True)
df.head()

## Explorando los datos

In [6]:
df.info()

### Valores faltantes

In [7]:
df.isnull().sum()

Como podemos ver, no tenemos ningún valor faltante, lo que es bueno.

### Distribución de las clases (0 real, 1 fraude)

In [8]:
print(df['fraude'].value_counts())
print(max(df['fraude'].value_counts().array) / (df['fraude'].count()))
df['fraude'].value_counts().plot(kind='bar')

Observemos que, como se mencionó en la introducción, tenemos un set de datos muy desbalanceado. Más del 99% de los datos pertenece a la clase 'real'.

## Tomando una muestra de transacciones legítimas del mismo tamaño que el de fraudulentas.

In [9]:
cantidad_fraudulentos = df.loc[df['fraude'] == 1].shape[0]

# Creando un dataframe separado con sólo transacciones reales
reales = df[ df['fraude'] == 0 ]
falsos = df[ df['fraude'] == 1 ]

# Tomando la muestra
muestra_reales = reales.sample(n = cantidad_fraudulentos)

# Uniendo las muestras
df_balanceado = pd.concat([falsos, muestra_reales], axis=0)

In [10]:
df_balanceado.head()

In [11]:
df_balanceado.tail()

In [12]:
df_balanceado['fraude'].value_counts()

Como ven, ahora tenemos un set de datos balanceado

## Separando las características de la variable objetivo

In [13]:
X = df_balanceado.drop(columns='fraude', axis = 1) # Dropeamos la columna fraude y nos quedamos sólo con las características
y = df_balanceado['fraude']  # Tomamos la columna de 'fraude' como nuestra variable objetivo

In [14]:
print(X)

In [15]:
print(y)

## Dividir los datos en subconjuntos de prueba y entrenamiento

In [16]:
# importamos las librerias necesarias
from sklearn.model_selection import train_test_split

In [46]:
# 80% para entrenamiento y 20% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEMILLA_ALEATORIA)

In [18]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Entrenando modelos

## Regresión Logística

In [19]:
# Librerias necesarias para el modelo y para medir la precisión
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
# Creamos nuestro modeo
modelo_rl = LogisticRegression(solver='liblinear')

In [21]:
# Entrenando el modelo de regresión logística con los datos de entrenamiento
modelo_rl.fit(X_train, y_train)

### Evaluando el modelo
#### Accuracy score para las muestras con las que fue entrenado

In [22]:
y_pred_train = modelo_rl.predict(X_train)
precision_rl_train = accuracy_score(y_train, y_pred_train)

# Imprimimos la precisión de entrenamiento
print('Precisión del modelo para el set de entrenamiento:', precision_rl_train)

#### Accuracy score para las muestras de prueba

In [23]:
y_pred_test = modelo_rl.predict(X_test)
precision_rl_test = accuracy_score(y_test, y_pred_test)

# Imprimimos la precisión de prueba
print('Precision del modelo para el set de prueba:', precision_rl_test)

#### Resultados del modelo de Regresión Logística
Obtuvimos una precisión del 92.88% para el set de entrenameinto.  
Obtuvimos una precisión el 95.93% para el set de prueba.  
Podemos decir que el modelo generalizó bastante bien utilizando solamente menos de 500 muestras de cada clase.

## Modelos basados en árboles

### Bosques aleatorios (random forest)

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

modelo_rf = RandomForestClassifier(max_depth=8, random_state=SEMILLA_ALEATORIA)
modelo_rf.fit(X_train, y_train)

modelo_rf_pred = modelo_rf.predict(X_test)
precision_rf = accuracy_score(y_test, modelo_rf_pred)

print('Precision del modelo de bosques aleatorios:', precision_rf)
# Matrix de confusion
print(confusion_matrix(y_test, modelo_rf_pred))

#### Resultados del modelo de bosque aleatorio
Hasta ahora, hemos obtenido la precisión más alta con bosque aleatorio, que fue de un 97.4%
Y como podemos observar en la matriz de confusión, tenemos una buena separación de los tipos de datos.

### Bosque de aislamiento (isolation forest)

In [55]:
from sklearn.ensemble import IsolationForest

modelo_if = IsolationForest(random_state=SEMILLA_ALEATORIA)
modelo_if.fit(X_train, y_train)

modelo_if_pred = modelo_if.predict(X_test)
precision_if = accuracy_score(y_test, modelo_if_pred)

print('Precision del modelo de bosque de aislamiento:', precision_if)

#### Resultados de bosques de aislamiento
Nos arrojaron una precision del 32%, lo que es bastante malo.

## Un acercamiento con autoencoders
### ¿Qué es un autoencoder?
Un autoencoder no es más que otra arquitectura de red neuronal que, en cierta fase de la misma comprime la información recibida a una versión muy reducida, denominado espacio latente. Al proceso de comprimir la información de una entrada al espacio latente, se le llama reducción de la dimensionalidad.  
La entrada y la salida de un autoencoder son la misma. ¿Por qué? porque queremos que nuestra red neuronal aprenda a reconstruir, con error mínimo, los datos que le son dados.    

**¿De qué sirve obtener como salida lo que alimento como entrada?**  
A simple vista, se puede creer que este resultado es poco útil, pero no es así, pues, al comparar los datos de entrada y los de salida podemos calcular un **error**, es decir, que tanto difiere la entrada de la salida.  
Esto nos sirve para aprovechar sets de datos muy debalanceados, pues la estrategia es entrenar al autoencoder para que aprenda a reconstruir entradas de una sola clase (de la que tengamos más muestras) con un error muy bajo. Con esto, cuando la entrada sea un dato anómalo, el error deberá ser mayor al error promedio que tendría reconstruyendo un dato normal.  
Es importante establecer un threshold adecuado para poder distinguir cuándo un error de reconstrucción dictará que el dato de entrada se trata de una anomalía.  
  
Intentemos utilizar un autoencoder para detectar cuando una transacción es fraudulenta.

### Normalizando los datos de la columna monto
Para normalizar los datos utilizaremos una distribución normal logarítmica equivalente.

In [25]:
# Añadimos una cantidad insignificante para evitar tomar el logaritmo de 0
df['log10_monto'] = np.log10(df.monto + 0.00001)

In [26]:
df.head()

In [27]:
RATIO_FRAUDE = 15

df2 = df.copy().drop(['tiempo', 'monto'], axis=1)

# Dividir por clase
fraude = df2[df2.fraude == 1]
real   = df2[df2.fraude == 0]

# Tomamos una muestra de las transacciones reales
muestra_reales = real.sample(int(len(fraude) * RATIO_FRAUDE), random_state=SEMILLA_ALEATORIA)

# Concatenamos los nuevos dataframes en uno solo
visualizacion = pd.concat([fraude, muestra_reales])
nombres_columnas = list(visualizacion.drop('fraude', axis=1).columns)

# Aislar las características de las etiquetas
caract, etiquetas = visualizacion.drop('fraude', axis=1).values , visualizacion.fraude.values

In [28]:
print(f"""The non-fraud dataset has been undersampled from {len(real):,} to {len(muestra_reales):,}.
This represents a ratio of {RATIO_FRAUDE}:1 to fraud.""")

### t-SNE

In [29]:
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D

def tsne_scatter(features, labels, dimensions=2, save_as='graph.png'):
    if dimensions not in (2, 3):
        raise ValueError('tsne_scatter can only plot in 2d or 3d (What are you? An alien that can visualise >3d?). Make sure the "dimensions" argument is in (2, 3)')

    # t-SNE dimensionality reduction
    features_embedded = TSNE(n_components=dimensions, random_state=SEMILLA_ALEATORIA).fit_transform(features)
    
    # initialising the plot
    fig, ax = plt.subplots(figsize=(8,8))
    
    # counting dimensions
    if dimensions == 3: ax = fig.add_subplot(111, projection='3d')

    # plotting data
    ax.scatter(
        *zip(*features_embedded[np.where(labels==1)]),
        marker='o',
        color='r',
        s=2,
        alpha=0.7,
        label='Fraude'
    )
    ax.scatter(
        *zip(*features_embedded[np.where(labels==0)]),
        marker='o',
        color='g',
        s=2,
        alpha=0.3,
        label='Real'
    )

    # storing it to be displayed later
    plt.legend(loc='best')
    plt.savefig(save_as);
    plt.show;

In [30]:
tsne_scatter(caract, etiquetas, dimensions=2, save_as='tsne_initial_2d.png')

### Separando en set de entrenamiento y prueba
Como mencionamos antes, los autoencoders solo se entrenan utilizando muestras de una clase. En este caso, entrenaremos al nuestro con las muestras de la clase de transacciones reales.

In [31]:
# Revolver el set de entrenamiento
real = real.sample(frac=1).reset_index(drop=True)

# Set de entrenamiento, solo transacciones no fraudulentas
X_train = real.iloc[:MUESTRA_ENTRENAMIENTO].drop('fraude', axis=1)

# Set de prueba, las muestras no fraudulentas que restan y todas las fraudulentas
X_test = real.iloc[MUESTRA_ENTRENAMIENTO:].append(fraude).sample(frac=1)

In [32]:
# Dividiendo el entrenamiento y la validación
X_train, X_validate = train_test_split(X_train, test_size=0.2, random_state=SEMILLA_ALEATORIA)

# Manualmente dividiendo las etiquetas del dataframe de prueba
X_test, y_test = X_test.drop('fraude', axis=1).values, X_test.fraude.values

### Normalizando los datos y creando un Pipeline
Normalizar ayuda al modelo a converger más rápido

In [33]:
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline

# Configurar el Pipeline
pipeline = Pipeline([('normalizer', Normalizer()), ('scaler', MinMaxScaler())])

#### Entrenando el pipeline

In [34]:
pipeline.fit(X_train)

#### Aplicamos transformaciones con los parámetros adquiridos

In [35]:
X_train_transformed = pipeline.transform(X_train)
X_validate_transformed = pipeline.transform(X_validate)

### Entrenando el Autoencoder

In [36]:
# Importando librerias necesarias
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Declaramos las constantes
tam_entrada = X_train_transformed.shape[1]
BATCH_SIZE = 256
EPOCHS = 100

# Creamos nuestra red neuronal
autoencoder = Sequential()
autoencoder.add( Dense(tam_entrada, activation='elu', input_shape=(tam_entrada,)) )
autoencoder.add( Dense(16, activation='elu') )
autoencoder.add( Dense(8, activation='elu') )
autoencoder.add( Dense(4, activation='elu') )
autoencoder.add( Dense(2, activation='elu') )

# Reconstrucción / decode
autoencoder.add( Dense(4, activation='elu') )
autoencoder.add( Dense(8, activation='elu') )
autoencoder.add( Dense(16, activation='elu') )
autoencoder.add( Dense(tam_entrada, activation='elu') )

autoencoder.compile(optimizer='adam', loss='mse', metrics=['acc'])

autoencoder.summary()

In [37]:
# EarlyStopping nos permite entrenar con muchos epochs y parar cuando nuestro modelo deje de aprender
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=10,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

### Entrenamiento

In [38]:
history = autoencoder.fit(X_train_transformed, X_train_transformed, 
                          shuffle=True, epochs=EPOCHS, batch_size=BATCH_SIZE, 
                          callbacks=[early_stopping],
                          validation_data=(X_validate_transformed, X_validate_transformed))

### Reconstrucciones

In [39]:
# Transformamos el set de prueba con los parámetros del pipeline
X_test_transformed = pipeline.transform(X_test)

# Pasamos los datos transformados a través del autoencoder para obtener el resultado de la reconstrucción
reconstrucciones = autoencoder.predict(X_test_transformed)

### Calculando la perdida de reconstrucción para cada transacción

In [40]:
# Calculando el error cuadrado medio de la reconstruccion
mse = np.mean(np.power(X_test_transformed - reconstrucciones, 2), axis=1)

In [41]:
fraude = mse[y_test == 0]
real   = mse[y_test == 1]

fig, ax = plt.subplots(figsize=(6,6))

ax.hist(real, bins=50, density=True, label='reales', alpha=0.6, color='green')
ax.hist(fraude, bins=50, density=True, label='fraudes', alpha=0.6, color='red')

plt.title('Distribución (normalizada) de la perdida de reconstrucción')
plt.legend()
plt.show()

Se ve que podríamos separar muchas transacciones fraudulentas de las reales, aunque algunas pasan por alto.

### Estableciendo un Threshold para clasificación

In [42]:
THRESHOLD = 3

def mad_score(points):
    m = np.median(points)
    ad = np.abs(points - m)
    mad = np.median(ad)
    
    return 0.6745 * ad / mad

z_scores = mad_score(mse)
outliers = z_scores > THRESHOLD

### Creando una matriz de confusión

In [44]:
from sklearn.metrics import confusion_matrix, precision_recall_curve

cm = confusion_matrix(y_test, outliers)
(tn, fp, fn, tp) = cm.flatten()

In [45]:
print(f"""La clasificación usando el metodo MAD con un threshold de ={THRESHOLD} son los siguientes:
{cm}

% de transacciones etiquetadas como fraude que fueron correctas (precisión): {tp}/({fp}+{tp}) = {tp/(fp+tp):.2%}
% de transacciones fraudulentas fueron detectadas satisfactoriamente (recall):    {tp}/({fn}+{tp}) = {tp/(fn+tp):.2%}""")

## Conclusión
El autoencoder no fue capaz de generalizar por completo. Sin embargo, tengamos en cuenta que este fue entrenado solo con una clase de transacciones, lo que es muy beneficioso para nosotros cuando tengamos un número muy mínimo de muestras de cierta clase.